In [7]:
import json
import re
import nltk
import emoji
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from wordcloud import WordCloud
from pysentimiento import SentimentAnalyzer
from pysentimiento.preprocessing import preprocess_tweet


In [8]:
with open('data/SENT-COVID.json') as file:
    data = json.load(file)
    
pd.options.mode.chained_assignment = None                                         
pd.set_option('display.max_colwidth',None)   


df = pd.DataFrame(data)
print('Numero de tweets: ' + str(len(df)))
df.head()

Numero de tweets: 4594


,Label,Tweet,id
0,NEUTRO,-@dulcema201 @BronstonRaqsa02 Protocolo de COVID !!!!,1401047081121353728
1,NEUTRO,-#COVID19 #QuedateEnCasa en Morelia Centro,1258159310162595843
2,POSITIVO,-México va en en aumento con el #Covid_19. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener 120 días en paro total. Pero podemos ser precavidos al usar la #SanaDistancia,1272748988626862082
3,NEUTRO,-@sororavirus Creo en todo y nada. 💜,1349385638722883585
4,NEGATIVO,-@GobiernoMX había prometido 389,1360615587114844161


In [20]:
def clean_tweet(text):
  text = re.sub(r'[~^0-9]', '', text) #numeros
  text = re.sub("\\s+", ' ', text) ##Espacios blancos dobles
  text = re.sub('\n', ' ', text) ##Saltos de linea

  pattern = r'([.])([A-Z#@¿])'
  pattern2 = r'([-])([a-zA-Z#@¿])'
  pattern3 = r'([a-zA-Z])([#@¿])'
  pattern4 = r'([:!])([a-zA-Z#@¿])'
  text = re.sub(pattern, r'\1 \2', text) # Separacion de punto seguido por una mayuscula
  text = re.sub(pattern2, r'\1 \2', text)
  text = re.sub(pattern3, r'\1 \2', text)
  text = re.sub(pattern4, r'\1 \2', text)
  return text 


def preprocess(text):
  return preprocess_tweet(text, shorten=2, emoji_wrapper='', user_token='@usuario')  # Preprocesamiento de pysentimiento   


def normalize(text):
 text = "".join(u for u in text if u not in ("?","¿", ".", ";", ":", "!","¡",'"',"%","“","”","$","&","'","\\", "(",")",
                                             "*","+",",","/","<",">","=","^","•","...", "ç","π","ⓘ", "-", "_","#","|"))
 a,b = 'áéíóúÁÉÍÓÚ','aeiouAEIOU'
 trans = str.maketrans(a,b)     
 text = text.translate(trans) # Reemplazo de palabras acentuadas       

 pattern = r'([a-z])([A-Z-])'
 text = re.sub(pattern, r'\1 \2', text)
 #text = re.sub(r'@[A-Za-z0-9_]+', '', text)
 text = text.lower()
 return text  


def tokenize(text):    
  text= text.split(sep = ' ')  # Tokenización por palabras individuales
  text= [token for token in text if len(token) > 1]  # Eliminación de tokens con una longitud < 2
  return(text) 

In [23]:
df['clean_tweet'] = df['Tweet'].apply(clean_tweet) 
df['preprocess_tweet'] = df['clean_tweet'].apply(preprocess)
df['norm_tweet'] = df['preprocess_tweet'].apply(normalize)
df['tokenized_tweet'] = df['norm_tweet'].apply(tokenize)

df[['Tweet','clean_tweet','norm_tweet','Label']][95:100]

,Tweet,clean_tweet,norm_tweet,Label
95,-#SNTEsalud ⚕️⚠️ ALERTA ⚠️ México vive alto contagio #coronavirus #COVID19 🦠 Ante síntomas no te automediques 💊🚫 ni tomes fármacos que prometen curar el #COVID 💊❌ y llama al 📲 800 00 44 800📱#QuedateEnCasa#FelizViernes #22DeMayo México #Sección35 #FelizFinde #RT https://t.co/wNIfhlJflO,- #SNTEsalud ⚕️⚠️ ALERTA ⚠️ México vive alto contagio #coronavirus #COVID 🦠 Ante síntomas no te automediques 💊🚫 ni tomes fármacos que prometen curar el #COVID 💊❌ y llama al 📲 📱#QuedateEnCasa #FelizViernes #DeMayo México #Sección #FelizFinde #RT https://t.co/wNIfhlJflO,sntesalud simbolo de medicina advertencia alerta advertencia mexico vive alto contagio coronavirus covid microbio ante sintomas no te automediques pildora prohibido ni tomes farmacos que prometen curar el covid pildora marca de cruz y llama al movil con una flecha telefono movil quedate en casa feliz viernes de mayo mexico seccion feliz finde rt url,NEGATIVO
96,-@LOVREGA @FelipeCalderon Pues como foca aplaudidora te queda bien el papel,- @LOVREGA @FelipeCalderon Pues como foca aplaudidora te queda bien el papel,@usuario @usuario pues como foca aplaudidora te queda bien el papel,NEGATIVO
97,"-""No habrá un """"regreso a la normalidad"""" en el mundo tras la pandemia de covid-19 https://t.co/bRO7munVtA vía @UniNoticias @dadams7308""","-""No habrá un """"regreso a la normalidad"""" en el mundo tras la pandemia de covid- https://t.co/bROmunVtA vía @UniNoticias @dadams""",no habra un regreso a la normalidad en el mundo tras la pandemia de covid url via @usuario @usuario,NEGATIVO
98,-Ya le hizo daño la vacuna.Cuando yo vivía en Alemania todo era paz y tranquilidad,- Ya le hizo daño la vacuna. Cuando yo vivía en Alemania todo era paz y tranquilidad,ya le hizo daño la vacuna cuando yo vivia en alemania todo era paz y tranquilidad,NEGATIVO
99,-Marcarle a mi preciosita en momento de crisis. 🥺🥺🥺,- Marcarle a mi preciosita en momento de crisis. 🥺🥺🥺,marcarle a mi preciosita en momento de crisis cara de por favor cara de por favor,POSITIVO


In [10]:
analyzer = SentimentAnalyzer(lang="es")


def polaridad(tweet):
    return analyzer.predict(tweet)


def prediction(x):
    sentence = Sentence(x)
    sia.predict(sentence)
    label = sentence.labels[0]
    labscore = (label.score)
    labvalue = (label.value)
    if labvalue == 'POSITIVE':
        response = labscore
    else:
        response = labscore*-1
    
    return response

Downloading: 100%|██████████| 841/841 [00:00<00:00, 263kB/s]
Downloading: 100%|██████████| 236k/236k [00:00<00:00, 833kB/s] 
Downloading: 100%|██████████| 469k/469k [00:00<00:00, 1.05MB/s]
Downloading: 100%|██████████| 67.0/67.0 [00:00<00:00, 25.0kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 40.6kB/s]
Downloading: 100%|██████████| 419M/419M [00:52<00:00, 8.38MB/s] 


In [24]:
df["polarity"] = df["norm_tweet"].apply(polaridad)

In [25]:
df['score'] = 0

for i in df.index:
    if df['polarity'][i].output == 'POS':
        df['score'][i] = 'POSITIVO'
    elif df['polarity'][i].output == 'NEU':
        df['score'][i] = 'NEUTRO'
    elif df['polarity'][i].output == 'NEG':
        df['score'][i] = 'NEGATIVO'
        
df[['Tweet','Label','polarity','score']].head(10)        

,Tweet,Label,polarity,score
0,-@dulcema201 @BronstonRaqsa02 Protocolo de COVID !!!!,NEUTRO,"SentimentOutput(output=NEU, probas={NEU: 0.999, POS: 0.001, NEG: 0.000})",NEUTRO
1,-#COVID19 #QuedateEnCasa en Morelia Centro,NEUTRO,"SentimentOutput(output=NEU, probas={NEU: 0.999, POS: 0.001, NEG: 0.000})",NEUTRO
2,-México va en en aumento con el #Covid_19. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener 120 días en paro total. Pero podemos ser precavidos al usar la #SanaDistancia,POSITIVO,"SentimentOutput(output=NEG, probas={NEG: 0.992, NEU: 0.007, POS: 0.001})",NEGATIVO
3,-@sororavirus Creo en todo y nada. 💜,NEUTRO,"SentimentOutput(output=NEU, probas={NEU: 0.998, NEG: 0.001, POS: 0.001})",NEUTRO
4,-@GobiernoMX había prometido 389,NEGATIVO,"SentimentOutput(output=NEU, probas={NEU: 0.976, NEG: 0.022, POS: 0.002})",NEUTRO
5,-Vacunación en Tonalá,POSITIVO,"SentimentOutput(output=NEU, probas={NEU: 0.999, NEG: 0.001, POS: 0.000})",NEUTRO
6,-No es cierto lo que dice @WHO,NEGATIVO,"SentimentOutput(output=NEG, probas={NEG: 0.992, NEU: 0.008, POS: 0.001})",NEGATIVO
7,-Participa en familia con esta entretenida actividad #MejorEnCasa Manda tu vídeo a la brevedad y llevaré uno de los premios.#aquiestamos #IED #QuedateEnCasa https://t.co/BQZTpxwca2,POSITIVO,"SentimentOutput(output=POS, probas={POS: 0.999, NEU: 0.001, NEG: 0.000})",POSITIVO
8,-@lalitogonzaalez Acabar con el Coronavirus.,POSITIVO,"SentimentOutput(output=NEU, probas={NEU: 0.997, NEG: 0.002, POS: 0.001})",NEUTRO
9,-#InternationalDanceDay #DID #ENCASA https://t.co/crIQOdoPgG,NEUTRO,"SentimentOutput(output=NEU, probas={NEU: 0.979, NEG: 0.012, POS: 0.009})",NEUTRO


In [26]:
df['agreement'] = 0

for i in df.index:
    if df['Label'][i] == df['score'] [i]:
        df['agreement'][i] = 'match'
    elif df['Label'][i]=='POSITIVO' and df['score'][i]=='NEGATIVO':
        df['agreement'][i] = 'opp'
    elif df['score'][i]=='POSITIVO'and df['Label'][i]=='NEGATIVO':
        df['agreement'][i] = 'opp'
    else:
        df['agreement'][i] = 'not match'
        
df['matrix_value'] = 0

for i in df.index:
    if df['score'][i] =='POSITIVO' and df['Label'] [i] =='POSITIVO':
        df['matrix_value'][i] = 'VP'
    elif df['score'][i] =='NEGATIVO' and df['Label'] [i] =='NEGATIVO':
        df['matrix_value'][i] = 'VN'
    elif df['score'][i] =='POSITIVO' and df['Label'] [i] !='POSITIVO':
        df['matrix_value'][i] = 'FP'
    elif df['score'][i] =='NEGATIVO' and df['Label'] [i] !='NEGATIVO': 
        df['matrix_value'][i] = 'FN'
        
df[['Tweet','Label','polarity','score','agreement','matrix_value']].head()          

,Tweet,Label,polarity,score,agreement,matrix_value
0,-@dulcema201 @BronstonRaqsa02 Protocolo de COVID !!!!,NEUTRO,"SentimentOutput(output=NEU, probas={NEU: 0.999, POS: 0.001, NEG: 0.000})",NEUTRO,match,0
1,-#COVID19 #QuedateEnCasa en Morelia Centro,NEUTRO,"SentimentOutput(output=NEU, probas={NEU: 0.999, POS: 0.001, NEG: 0.000})",NEUTRO,match,0
2,-México va en en aumento con el #Covid_19. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener 120 días en paro total. Pero podemos ser precavidos al usar la #SanaDistancia,POSITIVO,"SentimentOutput(output=NEG, probas={NEG: 0.992, NEU: 0.007, POS: 0.001})",NEGATIVO,opp,FN
3,-@sororavirus Creo en todo y nada. 💜,NEUTRO,"SentimentOutput(output=NEU, probas={NEU: 0.998, NEG: 0.001, POS: 0.001})",NEUTRO,match,0
4,-@GobiernoMX había prometido 389,NEGATIVO,"SentimentOutput(output=NEU, probas={NEU: 0.976, NEG: 0.022, POS: 0.002})",NEUTRO,not match,0


In [27]:
def accuracy(vp, vn, fp, fn):
    ex = (vp+vn)/(vp+fp+fn+vn) #TP+TN/TP+FP+FN+TN
    return ex

def precision(vp, fp):
    pr = vp/(vp+fp) #TP/TP+FP
    return pr

def recall(vp, fn):
    se = vp/(vp+fn) #TP/TP+FN
    return se

def f1(vn, fp):
    es = 2*(recall(vp, fn)*precision(vp, fp)) / (recall(vp, fn)+precision(vp, fp))
    return es

vp=df['matrix_value'].str.contains('VP').value_counts()[True]
vn=df['matrix_value'].str.contains('VN').value_counts()[True]
fp=df['matrix_value'].str.contains('FP').value_counts()[True]
fn=df['matrix_value'].str.contains('FN').value_counts()[True]

In [28]:
print('Accuracy: '+ str(accuracy(vp, vn, fp, fn)))
print('Precision: '+ str(precision(vp, fp)))
print('Recall: '+ str(recall(vp, fn)))
print('f1-score: '+ str(f1(vn, fp)))

Accuracy: 0.6492335437330928
Precision: 0.58578856152513
Recall: 0.38540478905359177
f1-score: 0.4649243466299863


In [19]:
df['agreement'].value_counts()

match        2874
not match    1589
opp           131
Name: agreement, dtype: int64

In [14]:
df['matrix_value'].value_counts()

0     2272
VN    1278
FN     649
VP     286
FP      87
Name: matrix_value, dtype: int64

In [15]:
df['Label'].value_counts()

NEUTRO      2036
NEGATIVO    1560
POSITIVO     976
Name: Label, dtype: int64

In [16]:
df['score'].value_counts()

NEUTRO      2272
NEGATIVO    1927
POSITIVO     373
Name: score, dtype: int64

In [46]:

df['clean_tweet2'] = df['clean_tweet'].apply(preprocess_tweet)
df[['clean_tweet','clean_tweet2','Label','score',]].head(15)         

,clean_tweet,clean_tweet2,Label
0,Protocolo de COVID !!!!,Protocolo de COVID !!!,NEUTRO
1,COVID QuedateEnCasa en Morelia Centro,COVID QuedateEnCasa en Morelia Centro,NEUTRO
2,México va en en aumento con el Covid. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener días en paro total. Pero podemos ser precavidos al usar la SanaDistancia,México va en en aumento con el Covid. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener días en paro total. Pero podemos ser precavidos al usar la SanaDistancia,POSITIVO
3,Creo en todo y nada. 💜,Creo en todo y nada. emoji corazón morado emoji,NEUTRO
4,había prometido,había prometido,NEGATIVO
5,Vacunación en Tonalá,Vacunación en Tonalá,POSITIVO
6,No es cierto lo que dice,No es cierto lo que dice,NEGATIVO
7,Participa en familia con esta entretenida actividad MejorEnCasa Manda tu vídeo a la brevedad y llevaré uno de los premios.aquiestamos IED QuedateEnCasa,Participa en familia con esta entretenida actividad MejorEnCasa Manda tu vídeo a la brevedad y llevaré uno de los premios.aquiestamos IED QuedateEnCasa,POSITIVO
8,Acabar con el Coronavirus.,Acabar con el Coronavirus.,POSITIVO
9,InternationalDanceDay DID ENCASA,InternationalDanceDay DID ENCASA,NEUTRO


In [47]:
%%time

df["polarity2"] = df["clean_tweet2"].apply(polaridad)

In [48]:
df['score2'] = 0

for i in df.index:
    if df['polarity2'][i].output == 'POS':
        df['score2'][i] = 'POSITIVO'
    elif df['polarity2'][i].output == 'NEU':
        df['score2'][i] = 'NEUTRO'
    elif df['polarity2'][i].output == 'NEG':
        df['score2'][i] = 'NEGATIVO'
        
df[['clean_tweet', 'norm_tweet','Label','polarity2','score2']].head(10)    

KeyError: "['norm_tweet'] not in index"

In [49]:
df['agreement2'] = 0

for i in df.index:
    if df['Label'][i] == df['score2'] [i]:
        df['agreement2'][i] = 'match'
    elif df['Label'][i]=='POSITIVO' and df['score2'][i]=='NEGATIVO':
        df['agreement2'][i] = 'opp'
    elif df['score2'][i]=='POSITIVO'and df['Label'][i]=='NEGATIVO':
        df['agreement2'][i] = 'opp'
    else:
        df['agreement2'][i] = 'not match'
        
df['matrix_value2'] = 0

for i in df.index:
    if df['score2'][i] =='POSITIVO' and df['Label'] [i] =='POSITIVO':
        df['matrix_value2'][i] = 'VP'
    elif df['score2'][i] =='NEGATIVO' and df['Label'] [i] =='NEGATIVO':
        df['matrix_value2'][i] = 'VN'
    elif df['score2'][i] =='POSITIVO' and df['Label'] [i] !='POSITIVO':
        df['matrix_value2'][i] = 'FP'
    elif df['score2'][i] =='NEGATIVO' and df['Label'] [i] !='NEGATIVO': 
        df['matrix_value2'][i] = 'FN'
        
df[['clean_tweet2','Label','polarity2','score2','agreement2','matrix_value2']].head()     

,clean_tweet2,Label,polarity2,score2,agreement2,matrix_value2
0,Protocolo de COVID !!!,NEUTRO,"AnalyzerOutput(output=NEU, probas={NEU: 0.959, POS: 0.026, NEG: 0.015})",NEUTRO,match,0
1,COVID QuedateEnCasa en Morelia Centro,NEUTRO,"AnalyzerOutput(output=NEG, probas={NEG: 0.508, NEU: 0.468, POS: 0.024})",NEGATIVO,not match,FN
2,México va en en aumento con el Covid. Tal vez no tengamos la estabilidad de Europa o estados unidos. Para mantener días en paro total. Pero podemos ser precavidos al usar la SanaDistancia,POSITIVO,"AnalyzerOutput(output=NEG, probas={NEG: 0.986, NEU: 0.012, POS: 0.001})",NEGATIVO,opp,FN
3,Creo en todo y nada. emoji corazón morado emoji,NEUTRO,"AnalyzerOutput(output=NEU, probas={NEU: 0.964, POS: 0.033, NEG: 0.003})",NEUTRO,match,0
4,había prometido,NEGATIVO,"AnalyzerOutput(output=NEU, probas={NEU: 0.872, POS: 0.086, NEG: 0.042})",NEUTRO,not match,0


In [50]:
vp2=df['matrix_value2'].str.contains('VP').value_counts()[True]
vn2=df['matrix_value2'].str.contains('VN').value_counts()[True]
fp2=df['matrix_value2'].str.contains('FP').value_counts()[True]
fn2=df['matrix_value2'].str.contains('FN').value_counts()[True]

In [51]:
print('Accuracy: '+ str(accuracy(vp2, vn2, fp2, fn2)))
print('Precision: '+ str(precision(vp2, fp2)))
print('Recall: '+ str(recall(vp2, fn2)))
print('f1-score: '+ str(f1(vn2, fp2)))

Accuracy: 0.6797044763146458
Precision: 0.7667560321715817
Recall: 0.3055555555555556
f1-score: 0.4373088685015291
